In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
layers = keras.layers

# Code for calculating LE

In [5]:
def rnn_jac(Wxh, Whh, ht, xt, phiprime):
    """
    Compute the Jacobian of the RNN with respect to the hidden state ht
    :param Wxh: input-to-hidden weight matrix (U)
    :param Whh: hidden-to-hidden weight matrix (V)
    :param ht: current hidden state
    :param xt: current input
    :param phiprime: function handle for the derivative of the activation function
    :return: Jacobian matrix
    """
    ht = np.reshape(ht, [-1, 1])  # shape: (32, 1)
    xt = np.reshape(xt, [-1, 1])  # shape: (32, 1)
    # Compute the Jacobian of the RNN with respect to ht


    alpha=Wxh@xt + Whh@ht
    J=np.diag(phiprime(alpha).flatten())@Whh
    return J

def calc_LEs(x_batches, h0, RNNlayer, activation_function_prim=lambda x:np.heaviside(x,1), k_LE=1000):
    """
    Calculate the Lyapunov exponents of a batch of sequences using the QR method.
    :param x_batches: input sequences (batch_size, T, input_size)
    :param h0: initial hidden state (batch_size, hidden_size)
    :param RNNlayer: RNN layer object (e.g., tf.keras.layers.SimpleRNN)
    :param activation_function_prim: function handle to derivative of activation function used in the RNN layer
    :param k_LE: number of Lyapunov exponents to compute
    :return: Lyapunov exponents for each batch (batch_size, k_LE)
    """
    #get dimensions
    batch_size, hidden_size = h0.shape
    batch_sizeX, T, input_size = x_batches.shape
    if batch_size != batch_sizeX:
        raise ValueError("batch size of h and X not compatible")
    L = hidden_size

    #get recurrent cell
    RNNcell=RNNlayer.cell

    # Choose how many exponents to track
    k_LE = max(min(L, k_LE), 1)

    #save average Lyapunov exponent over the sequence for each batch
    lyaps_batches = np.zeros((batch_size, k_LE))
    #Loop over input sequence
    for batch in range(batch_size):
        x=x_batches[batch]
        ht=h0[batch]
        #Initialize Q
        Q = tf.eye(L)
        #keep track of average lyapunov exponents
        cum_lyaps = tf.zeros((k_LE,))

        for t in range(T):
            #Get next state ht+1 by taking a reccurent step
            xt=x[t]
            _, ht = RNNcell(xt, ht)

            #Get jacobian J
            Wxh, Whh, b = rnn_layer.get_weights()
            # Transpose to match math-style dimensions
            Wxh = Wxh.T  # Now shape (units, input_dim)
            Whh = Whh.T  # Now shape (units, units)
            J = rnn_jac(Wxh, Whh, ht, xt, activation_function_prim)
            #Get the Lyapunov exponents from qr decomposition
            Q=Q@J
            Q,R=tf.linalg.qr(Q, full_matrices=False)
            cum_lyaps += tf.math.log(tf.math.abs(tf.linalg.diag_part(R[0:k_LE, 0:k_LE])))
        lyaps_batches[batch] = cum_lyaps / T
    return lyaps_batches


# Code used to test/show implementation

Start out with defining and training a toy model

In [3]:
def define_model():
    """Define and compile a simple RNN model."""
    z0 = layers.Input(shape=[None, 2])  # time steps unspecified, 2 features
    z = layers.SimpleRNN(32, activation="tanh")(z0)
    z = layers.Dense(32, activation='relu')(z)
    z = layers.Dense(16, activation='relu')(z)
    z = layers.Dense(1)(z)

    model = keras.models.Model(inputs=z0, outputs=z)
    model.compile(loss='mse', optimizer='adam')
    return model

def train_model(model, X, y, epochs=20, batch_size=10):
    """Train model with early stopping and LR scheduler."""
    results = model.fit(
        X, y,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        verbose=1,
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(factor=0.67, patience=3, verbose=1, min_lr=1E-5),
            keras.callbacks.EarlyStopping(patience=4, verbose=1)
        ]
    )
    return results

# Create some toy data
n_samples = 300
time_steps = 20
X = np.random.rand(n_samples, time_steps, 2)  # [batch, time, features]
Y = np.random.rand(n_samples)

# Create and train model
model = define_model()
results = train_model(model, X, Y)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.1614 - val_loss: 0.1215 - learning_rate: 0.0010
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1052 - val_loss: 0.1117 - learning_rate: 0.0010
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0835 - val_loss: 0.1078 - learning_rate: 0.0010
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0769 - val_loss: 0.1391 - learning_rate: 0.0010
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0816 - val_loss: 0.1039 - learning_rate: 0.0010
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0795 - val_loss: 0.0903 - learning_rate: 0.0010
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0920 - val_loss: 0.0967 - learning_rate: 0.0010
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0755 - val_loss: 0.1112 - learning_rate: 0.0010
Epoch 9/20
24/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0715
Epoch 9: ReduceLROnPlateau reducing learning rate to 0.000670000031823

Now we can calulate the LEs of the model

In [7]:
#create some batches of input data and initial hidden states
batch_size = 10   # number of sequences
T = 20            # length of each sequence
input_dim = 2     # size of each x
hidden_dim = 32   # size of hidden state

X = np.random.rand(batch_size, T, input_dim)
H0 = np.random.rand(batch_size, hidden_dim)

#Get the rnn layer of the model
rnn_layer=model.layers[1]

#Define the derivative of the activation function used
tanh_prim=lambda x: 1-np.pow(np.tanh(x), 2)

#calculate the LEs
number_exponents=20
LEs=calc_LEs(X,H0, rnn_layer, tanh_prim, number_exponents)
#LEs[batch, exponent], exponents are not ordered
print(LEs)

[[-0.12576437 -0.14891843 -0.15324934 -0.14134982 -0.13428327 -0.12964118
  -0.10776679 -0.15571843 -0.11693958 -0.09222397 -0.16342787 -0.12318116
  -0.16189055 -0.13907412 -0.18277627 -0.17552233 -0.17539442 -0.17022833
  -0.21283035 -0.10185456]
 [-0.11316749 -0.13745716 -0.14568643 -0.13207491 -0.11823563 -0.11703672
  -0.09598998 -0.13540055 -0.11208098 -0.08343199 -0.16282794 -0.11166426
  -0.15727566 -0.12789448 -0.16299987 -0.16356137 -0.16367623 -0.16035384
  -0.19379027 -0.10163222]
 [-0.13708967 -0.13387711 -0.13818024 -0.13338847 -0.12198553 -0.13415112
  -0.11041255 -0.14069079 -0.12441449 -0.07649969 -0.15082738 -0.13414796
  -0.15845719 -0.12535499 -0.16645148 -0.15997513 -0.15692392 -0.17359036
  -0.19508681 -0.11404215]
 [-0.11866204 -0.14052059 -0.13954674 -0.12851325 -0.11859329 -0.11794174
  -0.09673239 -0.13649654 -0.10521861 -0.07816082 -0.13795233 -0.11968331
  -0.14843284 -0.12963128 -0.16704577 -0.1618019  -0.15577599 -0.15952009
  -0.19896838 -0.10463446]
 [-0